In [1]:
import glob
from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import numpy as np
import datetime
import pymysql
import json

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_colwidth', 50)

In [2]:
# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("./*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [6]:
sql = f"""
SELECT     concat(concat(cast(format("%02d",extract(hour from timestamp_micros(event_timestamp))) as string)),
format("%02d",extract(minute from timestamp_micros(event_timestamp)))) as date_hour_and_minute,
 * FROM `plasma-canyon-.analytics_.events_*`
"""

# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
df = query_job.to_dataframe()
df.head()

,date_hour_and_minute,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,...,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,0430,20221105,1667622634702710,session_start,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,-1497184394,<NA>,None,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[]
1,0430,20221105,1667622634702710,page_view,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,-1497184394,<NA>,None,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[]
2,0430,20221105,1667622634712033,page_view,"[{'key': 'source', 'value': {'string_value': '...",<NA>,NaN,-1497175071,<NA>,cl8vecvhf0000mo09b5vg1lkq,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[]
3,0430,20221105,1667622638999777,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,-1492887327,<NA>,cl8vecvhf0000mo09b5vg1lkq,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[]
4,0430,20221105,1667622658095193,page_view,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,-1473791911,<NA>,cl8vecvhf0000mo09b5vg1lkq,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[]


In [7]:
df.groupby(['event_date'])['event_name'].count().reset_index().sort_values(by=['event_date'], ascending=False)[:10]

,event_date,event_name
60,20230101,181
59,20221231,117
58,20221230,97
57,20221229,211
56,20221228,94
55,20221227,116
54,20221226,222
53,20221225,46
52,20221224,26
51,20221223,51


In [8]:
df.groupby(['event_date'])['user_id'].nunique().reset_index().sort_values(by=['event_date'], ascending=False)[:10]

,event_date,user_id
60,20230101,3
59,20221231,0
58,20221230,2
57,20221229,2
56,20221228,4
55,20221227,3
54,20221226,4
53,20221225,0
52,20221224,1
51,20221223,2


In [9]:
df.groupby(['event_date'])['user_pseudo_id'].nunique().reset_index().sort_values(by=['event_date'], ascending=False)[:10]

,event_date,user_pseudo_id
60,20230101,13
59,20221231,11
58,20221230,9
57,20221229,11
56,20221228,9
55,20221227,7
54,20221226,9
53,20221225,3
52,20221224,3
51,20221223,8


In [12]:
df['event_name'].value_counts()

page_view          7545
scroll             1790
user_engagement    1315
session_start       922
first_visit         343
click                61
filter_change        28
Name: event_name, dtype: int64

In [13]:
# df[df['event_name'].str.contains('filter_change')]['event_params'].to_csv('./filter_name.csv', encoding='CP949', index=False)

In [14]:
df['time'] =""

for i in df.index:
    df3 = pd.json_normalize(df['event_params'][i])
#     print(df3)
    df['time'][i] = df3[df3['key'] == 'engagement_time_msec']['value.int_value'].astype(str).str.split('Name:').str[0]
    del df3
    
df['time'] = df['time'].astype(str).str.split('Name:').str[0]
df['time'] = df['time'].astype(str).str.split('\n').str[0]
df['time'] = df['time'].astype(str).str.split('    ').str[1]
df['time'] = df['time'].astype(float) / 1000
df

,date_hour_and_minute,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,...,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,time
0,1500,20221112,1668178837607738,page_view,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,654939450,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],NaN
1,1500,20221112,1668178837678806,page_view,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655010518,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],0.005
2,1500,20221112,1668178837681793,scroll,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655013505,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],0.008
3,1500,20221112,1668178837340916,user_engagement,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,654672628,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],156.677
4,1500,20221112,1668178840680761,user_engagement,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,658012473,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'country': 'South Korea'...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],3.047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11999,1423,20221125,1669386235699293,scroll,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,1167220829,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Europe', 'country': 'Ireland', ...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN
12000,1423,20221125,1669386235699400,first_visit,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,1167220936,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Europe', 'country': 'Ireland', ...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN
12001,1423,20221125,1669386235699400,session_start,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,1167220936,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Europe', 'country': 'Ireland', ...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN
12002,1423,20221125,1669386235699400,page_view,"[{'key': 'page_referrer', 'value': {'string_va...",<NA>,NaN,1167220936,<NA>,None,...,"{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Europe', 'country': 'Ireland', ...",None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN


In [15]:
df['refer_page'] = ""
df['page_url'] =""

for i in df.index:
    df3 = pd.json_normalize(df['event_params'][i])
#     print(df3)
    df['refer_page'][i] = df3[df3['key'] == 'page_referrer']['value.string_value']
    df['page_url'][i] = df3[df3['key'] == 'page_location']['value.string_value']
#     df['time'][i] = df3[df3['key'] == 'engagement_time_msec']['value.int_value'].astype(str).str.split('Name:').str[0]
    del df3
    
df

,date_hour_and_minute,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,...,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,time,refer_page,page_url
0,1500,20221112,1668178837607738,page_view,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,654939450,<NA>,None,...,None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],NaN,3 https://zipbiseo.com/apartment/817?unit-t...,6 https://zipbiseo.com/signin?callbackUrl=h...
1,1500,20221112,1668178837678806,page_view,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655010518,<NA>,None,...,None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],0.005,1 https://zipbiseo.com/apartment/817?unit-t...,3 https://zipbiseo.com/signin?callbackUrl=h...
2,1500,20221112,1668178837681793,scroll,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655013505,<NA>,None,...,None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],0.008,8 https://zipbiseo.com/apartment/817?unit-t...,4 https://zipbiseo.com/signin?callbackUrl=h...
3,1500,20221112,1668178837340916,user_engagement,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,654672628,<NA>,None,...,None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],156.677,8 https://zipadvisor.co/ Name: value.string...,5 https://zipbiseo.com/apartment/817?unit-t...
4,1500,20221112,1668178840680761,user_engagement,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,658012473,<NA>,None,...,None,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],3.047,3 https://zipbiseo.com/apartment/817?unit-t...,6 https://zipbiseo.com/signin?callbackUrl=h...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11999,1423,20221125,1669386235699293,scroll,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,1167220829,<NA>,None,...,None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,4 https://www.facebook.com/ Name: value.str...,7 https://zipbiseo.com/?fbclid=IwAR1wE4S5yX...
12000,1423,20221125,1669386235699400,first_visit,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,1167220936,<NA>,None,...,None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,2 https://www.facebook.com/ Name: value.str...,3 https://zipbiseo.com/?fbclid=IwAR1wE4S5yX...
12001,1423,20221125,1669386235699400,session_start,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,1167220936,<NA>,None,...,None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,2 https://www.facebook.com/ Name: value.str...,5 https://zipbiseo.com/?fbclid=IwAR1wE4S5yX...
12002,1423,20221125,1669386235699400,page_view,"[{'key': 'page_referrer', 'value': {'string_va...",<NA>,NaN,1167220936,<NA>,None,...,None,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,0 https://www.facebook.com/ Name: value.str...,10 https://zipbiseo.com/?fbclid=IwAR1wE4S5y...


In [18]:
df['page_url'] = df['page_url'].astype(str).str.split(' ').str[4]
df['page_url'] = df['page_url'].astype(str)
df.loc[df['refer_page'].astype(str).str.contains('/filter'), 'filter_yn'] = 5
df

,date_hour_and_minute,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,...,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,time,refer_page,page_url,filter_yn
0,1500,20221112,1668178837607738,page_view,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,654939450,<NA>,None,...,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],NaN,3 https://zipbiseo.com/apartment/817?unit-t...,https://zipbiseo.com/signin?callbackUrl=https%...,NaN
1,1500,20221112,1668178837678806,page_view,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655010518,<NA>,None,...,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],0.005,1 https://zipbiseo.com/apartment/817?unit-t...,https://zipbiseo.com/signin?callbackUrl=https%...,NaN
2,1500,20221112,1668178837681793,scroll,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655013505,<NA>,None,...,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],0.008,8 https://zipbiseo.com/apartment/817?unit-t...,https://zipbiseo.com/signin?callbackUrl=https%...,NaN
3,1500,20221112,1668178837340916,user_engagement,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,654672628,<NA>,None,...,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],156.677,8 https://zipadvisor.co/ Name: value.string...,https://zipbiseo.com/apartment/817?unit-type-i...,NaN
4,1500,20221112,1668178840680761,user_engagement,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,658012473,<NA>,None,...,"{'name': '(direct)', 'medium': '(none)', 'sour...",4129127356,WEB,None,None,[],3.047,3 https://zipbiseo.com/apartment/817?unit-t...,https://zipbiseo.com/signin?callbackUrl=https%...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11999,1423,20221125,1669386235699293,scroll,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,1167220829,<NA>,None,...,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,4 https://www.facebook.com/ Name: value.str...,https://zipbiseo.com/?fbclid=IwAR1wE4S5yXJlS9b...,NaN
12000,1423,20221125,1669386235699400,first_visit,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,1167220936,<NA>,None,...,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,2 https://www.facebook.com/ Name: value.str...,https://zipbiseo.com/?fbclid=IwAR1wE4S5yXJlS9b...,NaN
12001,1423,20221125,1669386235699400,session_start,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,1167220936,<NA>,None,...,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,2 https://www.facebook.com/ Name: value.str...,https://zipbiseo.com/?fbclid=IwAR1wE4S5yXJlS9b...,NaN
12002,1423,20221125,1669386235699400,page_view,"[{'key': 'page_referrer', 'value': {'string_va...",<NA>,NaN,1167220936,<NA>,None,...,"{'name': '(referral)', 'medium': 'referral', '...",4129127356,WEB,None,None,[],NaN,0 https://www.facebook.com/ Name: value.str...,https://zipbiseo.com/?fbclid=IwAR1wE4S5yXJlS9b...,NaN


In [19]:
def value_emp(x):
    if x.empty == True:
        return 0
    
    elif (x.empty == False) & (type(x) == pd.core.series.Series):
        return str(x).split('    ')[1].split('\n')[0]

In [20]:
df['filter_tradetype'] =""
df['filter_price_max'] =""
df['filter_price_min'] =""
df['filter_unit_max'] =""
df['filter_unit_min'] =""
df['filter_jeonse_rate_max'] =""
df['filter_jeonse_rate_min'] =""
df['filter_room_max'] =""
df['filter_room_min'] =""
df['filter_bathroom_max'] =""
df['filter_bathroom_min'] =""
df['filter_gross_area_max'] =""
df['filter_gross_area_min'] =""
df['filter_built_year_month_max'] =""
df['filter_built_year_month_min'] =""


for i in df.index:
    df_fill = pd.json_normalize(df['event_params'][i])
#     print(df3)
    df['filter_tradetype'][i] = value_emp(df_fill[df_fill['key'] == 'tradeType']['value.string_value'].astype(str))
    df['filter_price_max'][i] = value_emp(df_fill[df_fill['key'] == 'price_max']['value.int_value'].astype(str))
    df['filter_price_min'][i] = value_emp(df_fill[df_fill['key'] == 'price_min']['value.int_value'].astype(str))
    
    df['filter_unit_max'][i] = value_emp(df_fill[df_fill['key'] == 'unit_max']['value.int_value'].astype(str))
    df['filter_unit_min'][i] = value_emp(df_fill[df_fill['key'] == 'unit_min']['value.int_value'].astype(str))

    df['filter_jeonse_rate_max'][i] = value_emp(df_fill[df_fill['key'] == 'jeonse_rate_max']['value.double_value'].astype(str))
    df['filter_jeonse_rate_min'][i] = value_emp(df_fill[df_fill['key'] == 'jeonse_rate_min']['value.double_value'].astype(str))

    df['filter_room_max'][i] = value_emp(df_fill[df_fill['key'] == 'room_max']['value.int_value'].astype(str))
    df['filter_room_min'][i] = value_emp(df_fill[df_fill['key'] == 'room_min']['value.int_value'].astype(str))
    
    df['filter_bathroom_max'][i] = value_emp(df_fill[df_fill['key'] == 'bathroom_max']['value.int_value'].astype(str))
    df['filter_bathroom_min'][i] = value_emp(df_fill[df_fill['key'] == 'bathroom_min']['value.int_value'].astype(str))
    
    df['filter_gross_area_max'][i] = value_emp(df_fill[df_fill['key'] == 'gross_area_max']['value.int_value'].astype(str))
    df['filter_gross_area_min'][i] = value_emp(df_fill[df_fill['key'] == 'gross_area_min']['value.int_value'].astype(str))
    
    df['filter_built_year_month_max'][i] = value_emp(df_fill[df_fill['key'] == 'built_year_month_max']['value.int_value'].astype(str))
    df['filter_built_year_month_min'][i] = value_emp(df_fill[df_fill['key'] == 'built_year_month_min']['value.int_value'].astype(str))
    
    del df_fill

df

,date_hour_and_minute,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,...,filter_jeonse_rate_max,filter_jeonse_rate_min,filter_room_max,filter_room_min,filter_bathroom_max,filter_bathroom_min,filter_gross_area_max,filter_gross_area_min,filter_built_year_month_max,filter_built_year_month_min
0,1500,20221112,1668178837607738,page_view,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,654939450,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
1,1500,20221112,1668178837678806,page_view,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655010518,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
2,1500,20221112,1668178837681793,scroll,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,655013505,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
3,1500,20221112,1668178837340916,user_engagement,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,654672628,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
4,1500,20221112,1668178840680761,user_engagement,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,658012473,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11999,1423,20221125,1669386235699293,scroll,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,1167220829,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
12000,1423,20221125,1669386235699400,first_visit,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,1167220936,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
12001,1423,20221125,1669386235699400,session_start,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,1167220936,<NA>,None,...,0,0,0,0,0,0,0,0,0,0
12002,1423,20221125,1669386235699400,page_view,"[{'key': 'page_referrer', 'value': {'string_va...",<NA>,NaN,1167220936,<NA>,None,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df3 = df[df['page_url'].astype(str).str.contains('apartment')]

df3_map = df3[df3['page_url'].astype(str).str.contains('map')]
df3_nomap = df3[~df3['page_url'].astype(str).str.contains('map')]
# df3['apartment'] = df3['page_url'].astype(str).str.split('apartment')
df3_map['url_1'] = df3_map['page_url'].astype(str).str.split('&').str[0]
df3_map['kbm_apartment_id'] = df3_map['url_1'].astype(str).str.split('=').str[1]

df3_map = df3_map[(df3_map['kbm_apartment_id'].isnull() == False) & (df3_map['kbm_apartment_id'] != '')]
df3_map = df3_map[~(df3_map['kbm_apartment_id'].str.contains('Name'))]
df3_map

,date_hour_and_minute,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,...,filter_room_max,filter_room_min,filter_bathroom_max,filter_bathroom_min,filter_gross_area_max,filter_gross_area_min,filter_built_year_month_max,filter_built_year_month_min,url_1,kbm_apartment_id
48,0910,20221112,1668244253518611,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,1646340883,<NA>,None,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=29602,29602
54,2354,20221112,1668210888045307,page_view,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,-1654361349,<NA>,None,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=29602,29602
56,2356,20221112,1668210992522345,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,-1549884311,<NA>,None,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=29602,29602
163,0259,20221205,1670209187359701,page_view,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-514839595,<NA>,clba7eici000aqsoaangf9wpi,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=1367,1367
165,0300,20221205,1670209231312777,page_view,"[{'key': 'page_referrer', 'value': {'string_va...",<NA>,NaN,-470886519,<NA>,clba7eici000aqsoaangf9wpi,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=1372,1372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877,0112,20221125,1669338729688113,user_engagement,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,905849905,<NA>,None,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=47302,47302
11904,0112,20221125,1669338729688113,session_start,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,905849905,<NA>,None,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=47302,47302
11941,0030,20221125,1669336221857154,user_engagement,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-1601981054,<NA>,None,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=1915,1915
11974,0131,20221125,1669339873597575,page_view,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,2049759367,<NA>,None,...,0,0,0,0,0,0,0,0,https://zipbiseo.com/map?apartment-id=7424,7424


In [24]:
df4 = df3_map[df3_map['user_id'].isnull() == False][[ 'user_id',  'event_name', 'kbm_apartment_id']]
df4['apt_view_cnt'] = df4.groupby(['user_id', 'kbm_apartment_id'])['event_name'].transform('count')
df4

,user_id,event_name,kbm_apartment_id,apt_view_cnt
163,clba7eici000aqsoaangf9wpi,page_view,1367,1
165,clba7eici000aqsoaangf9wpi,page_view,1372,1
167,clba7eici000aqsoaangf9wpi,page_view,8244,4
169,clba7eici000aqsoaangf9wpi,page_view,8244,4
171,clba7eici000aqsoaangf9wpi,page_view,2813,1
...,...,...,...,...
11651,cl8vem63w0000mh09b8zbsclp,page_view,31983,2
11789,cl8vem63w0000mh09b8zbsclp,page_view,32148,3
11791,cl8vem63w0000mh09b8zbsclp,page_view,32148,3
11793,cl8vem63w0000mh09b8zbsclp,page_view,32148,3


In [27]:
df4_2.groupby(['user_id', '시/도'])['event_name'].count().reset_index()

,user_id,시/도,event_name
0,cl8vecvhf0000mo09b5vg1lkq,서울특별시,8
1,cl8vem63w0000mh09b8zbsclp,서울특별시,9
2,cl8wejtlj0000l309v0qc88kc,경기도,4
3,cl8wejtlj0000l309v0qc88kc,서울특별시,89
4,cl9wdu2430000ld09m0clpn4x,서울특별시,17
5,cl9wdu2430000ld09m0clpn4x,인천광역시,1
6,cla1vglew000sqs9nigwz07tk,서울특별시,1
7,clag36ol60009qs6017o0mmq1,서울특별시,2
8,claltvb7k0000qsf2lg6yr9gv,서울특별시,8
9,claobq8050009qsf2q0jlad2r,서울특별시,2


In [28]:
df4_2.groupby(['user_id', '시/도', '시군구'])['event_name'].count().reset_index().to_csv('./시군구별조회수.csv', encoding='CP949', index=False)

In [29]:
df4_2.groupby(['user_id', '시/도', '시군구'])['event_name'].count().reset_index()

,user_id,시/도,시군구,event_name
0,cl8vecvhf0000mo09b5vg1lkq,서울특별시,강남구,5
1,cl8vecvhf0000mo09b5vg1lkq,서울특별시,영등포구,3
2,cl8vem63w0000mh09b8zbsclp,서울특별시,강남구,3
3,cl8vem63w0000mh09b8zbsclp,서울특별시,서초구,3
4,cl8vem63w0000mh09b8zbsclp,서울특별시,송파구,3
5,cl8wejtlj0000l309v0qc88kc,경기도,남양주시,4
6,cl8wejtlj0000l309v0qc88kc,서울특별시,강남구,14
7,cl8wejtlj0000l309v0qc88kc,서울특별시,강동구,12
8,cl8wejtlj0000l309v0qc88kc,서울특별시,서초구,53
9,cl8wejtlj0000l309v0qc88kc,서울특별시,송파구,6


In [30]:
df4_2.groupby(['user_id', '시/도', '시군구', '법정동'])['event_name'].count().reset_index().to_csv('./법정동별조회수.csv', encoding='CP949', index=False)

In [31]:
df4_2.groupby(['user_id', '시/도', '시군구', '법정동'])['event_name'].count().reset_index()

,user_id,시/도,시군구,법정동,event_name
0,cl8vecvhf0000mo09b5vg1lkq,서울특별시,강남구,삼성동,5
1,cl8vecvhf0000mo09b5vg1lkq,서울특별시,영등포구,여의도동,3
2,cl8vem63w0000mh09b8zbsclp,서울특별시,강남구,삼성동,3
3,cl8vem63w0000mh09b8zbsclp,서울특별시,서초구,반포동,2
4,cl8vem63w0000mh09b8zbsclp,서울특별시,서초구,서초동,1
5,cl8vem63w0000mh09b8zbsclp,서울특별시,송파구,가락동,3
6,cl8wejtlj0000l309v0qc88kc,경기도,남양주시,와부읍,4
7,cl8wejtlj0000l309v0qc88kc,서울특별시,강남구,논현동,2
8,cl8wejtlj0000l309v0qc88kc,서울특별시,강남구,삼성동,8
9,cl8wejtlj0000l309v0qc88kc,서울특별시,강남구,청담동,4


In [32]:
df4_1 = df4.groupby(['user_id','event_name'])['kbm_apartment_id'].count().reset_index()
df4_1 = df4_1.rename(columns={'kbm_apartment_id' : 'event_cnt'})
df4_1

,user_id,event_name,event_cnt
0,cl8vecvhf0000mo09b5vg1lkq,page_view,7
1,cl8vecvhf0000mo09b5vg1lkq,session_start,1
2,cl8vem63w0000mh09b8zbsclp,page_view,8
3,cl8vem63w0000mh09b8zbsclp,session_start,1
4,cl8wejtlj0000l309v0qc88kc,page_view,84
5,cl8wejtlj0000l309v0qc88kc,session_start,9
6,cl9wdu2430000ld09m0clpn4x,page_view,18
7,cla1vglew000sqs9nigwz07tk,page_view,1
8,clag36ol60009qs6017o0mmq1,page_view,2
9,claltvb7k0000qsf2lg6yr9gv,page_view,8


In [36]:
df3_map_re[['user_id','kbm_apartment_id','grade']].sort_values(by=['grade'], ascending = False).to_csv('./사용자 행동기반 평가 데이터 샘플.csv', encoding = 'CP949', index = False)
df3_map_re[['user_id','kbm_apartment_id','grade']].sort_values(by=['grade'], ascending = False)

,user_id,kbm_apartment_id,grade
31,cl8wejtlj0000l309v0qc88kc,44285,52.8624
33,cl8wejtlj0000l309v0qc88kc,19763,51.3756
58,cl8wejtlj0000l309v0qc88kc,32109,41.1830
92,cl8wejtlj0000l309v0qc88kc,2171,32.6051
16,cl8wejtlj0000l309v0qc88kc,331,29.4030
...,...,...,...
144,clbn5c9yp001qqsoarf5tjqn8,1661,1.0041
143,clbn5c9yp001qqsoarf5tjqn8,26135,1.0034
14,claltvb7k0000qsf2lg6yr9gv,68,1.0028
8,claltvb7k0000qsf2lg6yr9gv,15364,1.0027


In [ ]:
df3_map_re = pd.merge(df3_map, m_dt1, on=['user_id'], how ='inner')
df3_map_re = pd.merge(df3_map_re, m_dt11, on=['user_id','kbm_apartment_id'], how ='left')
# df3_map_re = pd.merge(df3_map_re, m_dt1, on=['user_id'], how ='inner')
df3_map_re = df3_map_re[['user_id','NAME', 'kbm_apartment_id','event_name', 'time','filter_yn', 'LIKE']]
df3_map_re.loc[df3_map_re.time > 60 ,'time'] = 60

df3_map_re['num_view'] = df3_map_re.groupby(['user_id', 'NAME', 'kbm_apartment_id'])['event_name'].transform('count')
df3_map_re['sum_time'] = df3_map_re.groupby(['user_id', 'NAME', 'kbm_apartment_id'])['time'].transform('sum') / 10

df3_map_re = df3_map_re[['user_id', 'NAME', 'kbm_apartment_id', 'filter_yn', 'LIKE', 'num_view', 'sum_time']].drop_duplicates()
df3_map_re = df3_map_re.fillna(0)
df3_map_re['grade'] = df3_map_re['num_view'] + df3_map_re['sum_time'] + df3_map_re['LIKE']

In [37]:
df3_nomap['url_1'] = df3_nomap['page_url'].astype(str).str.split('?').str[0]
df3_nomap['kbm_apartment_id'] = df3_nomap['url_1'].astype(str).str.split('apartment/').str[1]

df3_nomap['url_2'] = df3_nomap['page_url'].astype(str).str.split('?').str[1]
df3_nomap['url_3'] = df3_nomap['url_2'].astype(str).str.split('unit-type-id=').str[1]
df3_nomap['kbm_unit_type_id'] = df3_nomap['url_3'].astype(str).str.split('\n').str[0]
df3_nomap

,date_hour_and_minute,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,...,filter_bathroom_min,filter_gross_area_max,filter_gross_area_min,filter_built_year_month_max,filter_built_year_month_min,url_1,kbm_apartment_id,url_2,url_3,kbm_unit_type_id
3,1500,20221112,1668178837340916,user_engagement,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,654672628,<NA>,None,...,0,0,0,0,0,https://zipbiseo.com/apartment/817,817,unit-type-i...\nName:,NaN,nan
49,0910,20221112,1668244255775810,page_view,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,1648598082,<NA>,None,...,0,0,0,0,0,https://zipbiseo.com/apartment/29602,29602,unit-type...\nName:,NaN,nan
55,2354,20221112,1668210892207377,page_view,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,-1650199279,<NA>,None,...,0,0,0,0,0,https://zipbiseo.com/apartment/29602,29602,unit-type...\nName:,NaN,nan
66,1429,20221112,1668263360812390,page_view,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-721201818,<NA>,None,...,0,0,0,0,0,https://zipbiseo.com/apartment/29602,29602,unit-type...\nName:,NaN,nan
164,0300,20221205,1670209210730159,page_view,"[{'key': 'page_referrer', 'value': {'string_va...",<NA>,NaN,-491469137,<NA>,clba7eici000aqsoaangf9wpi,...,0,0,0,0,0,https://zipbiseo.com/apartment/1372,1372,unit-type-...\nName:,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11848,1006,20221109,1667988417940792,user_engagement,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,-786166472,<NA>,None,...,0,0,0,0,0,https://zipbiseo.com/apartment/29602,29602,unit-type...\nName:,NaN,nan
11859,1006,20221109,1667988404579266,page_view,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-799527998,<NA>,cla1vglew000sqs9nigwz07tk,...,0,0,0,0,0,https://zipbiseo.com/apartment/29602,29602,unit-type...\nName:,NaN,nan
11876,0658,20221125,1669359508578241,user_engagement,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,209903553,<NA>,None,...,0,0,0,0,0,https://zipbiseo.com/apartment/27396,27396,unit-type...\nName:,NaN,nan
11929,0658,20221125,1669359494299113,page_view,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,195624425,<NA>,None,...,0,0,0,0,0,https://zipbiseo.com/apartment/27396,27396,unit-type...\nName:,NaN,nan


In [43]:
df4['num_view'] = df4.groupby(['user_id', 'NAME', 'kbm_apartment_id'])['event_name'].transform('count')
df4['sum_time'] = df4.groupby(['user_id', 'NAME', 'kbm_apartment_id'])['time'].transform('sum') / 10

In [45]:
df_1 = df[['event_date', 'date_hour_and_minute', 'user_id', 'event_name', 'time','event_params', 'refer_page', 'page_url']]
df_2 = pd.merge(df_1, m_dt1, on=['user_id'], how = 'inner')
# df_2 = pd.merge(df_2, m_dt11, on=['user_id'], how = 'left')
df_2

,event_date,date_hour_and_minute,user_id,event_name,time,event_params,refer_page,page_url,NAME,email,phone_number,created_at
0,20221112,0350,cl958dhru0000li09ntfxl9pq,page_view,NaN,"[{'key': 'page_location', 'value': {'string_va...","Series([], Name: value.string_value, dtype: ob...",https://zipbiseo.com/mydata/report\nName:,남성태,stforyou@gmail.com,+82 10-2595-4448,2022-11-14 10:04:46
1,20221108,0126,cl958dhru0000li09ntfxl9pq,page_view,0.008,"[{'key': 'session_engaged', 'value': {'string_...",1 http://staging.zipadvisor.co/ Name: value...,https://zipbiseo.com/\nName:,남성태,stforyou@gmail.com,+82 10-2595-4448,2022-11-14 10:04:46
2,20221214,0201,cl958dhru0000li09ntfxl9pq,page_view,0.007,"[{'key': 'engagement_time_msec', 'value': {'st...","Series([], Name: value.string_value, dtype: ob...",https://zipbiseo.com/\nName:,남성태,stforyou@gmail.com,+82 10-2595-4448,2022-11-14 10:04:46
3,20221214,0201,cl958dhru0000li09ntfxl9pq,page_view,4.682,"[{'key': 'page_title', 'value': {'string_value...","Series([], Name: value.string_value, dtype: ob...",https://zipbiseo.com/mydata/report\nName:,남성태,stforyou@gmail.com,+82 10-2595-4448,2022-11-14 10:04:46
4,20221114,0505,cl958dhru0000li09ntfxl9pq,page_view,0.004,"[{'key': 'entrances', 'value': {'string_value'...","Series([], Name: value.string_value, dtype: ob...",https://zipbiseo.com/recommendation\nName:,남성태,stforyou@gmail.com,+82 10-2595-4448,2022-11-14 10:04:46
...,...,...,...,...,...,...,...,...,...,...,...,...
2033,20221117,0612,clakobin8000hqsc8mfqr1pg1,page_view,17.746,"[{'key': 'campaign', 'value': {'string_value':...",1 https://kauth.kakao.com/ Name: value.stri...,https://zipbiseo.com/map\nName:,아라,nanu2302@naver.com,+82 10-4057-1104,2022-11-17 15:10:41
2034,20221117,0745,clakobin8000hqsc8mfqr1pg1,session_start,NaN,"[{'key': 'page_referrer', 'value': {'string_va...",0 https://kauth.kakao.com/ Name: value.stri...,https://zipbiseo.com/apartment/30739?unit-type...,아라,nanu2302@naver.com,+82 10-4057-1104,2022-11-17 15:10:41
2035,20221117,0745,clakobin8000hqsc8mfqr1pg1,page_view,NaN,"[{'key': 'page_location', 'value': {'string_va...",3 https://kauth.kakao.com/ Name: value.stri...,https://zipbiseo.com/apartment/30739?unit-type...,아라,nanu2302@naver.com,+82 10-4057-1104,2022-11-17 15:10:41
2036,20221125,0232,clavw1mmc0010qsf2773wn05y,page_view,0.010,"[{'key': 'page_location', 'value': {'string_va...",3 https://kauth.kakao.com/ Name: value.stri...,https://zipbiseo.com/\nName:,왕태일,mrjamesjj@kakao.com,+82 10-7175-3437,2022-11-25 02:32:24


In [46]:
df_2['user_id'].value_counts()

cl8wejtlj0000l309v0qc88kc    472
claltvb7k0000qsf2lg6yr9gv    298
cl8vem63w0000mh09b8zbsclp    240
cl9wdu2430000ld09m0clpn4x    237
clbn5c9yp001qqsoarf5tjqn8    110
cl8vecvhf0000mo09b5vg1lkq     94
clbel4qum0015qsoamtw3ozlj     73
cl958dhru0000li09ntfxl9pq     71
cla0cv9o1000bqs9n6oryls1n     50
claqd05eo000fqsf2hhyrtpz0     49
clba7eici000aqsoaangf9wpi     39
cl9z1gws60007qs9naqx4n380     33
clbq7ocv0001xqsoar9zedylk     27
clbak7rqa000eqsoa9142bz6x     26
clbsol14v002hqsoaq3vf8fst     26
clbrobicr0022qsoaqa7w410o     20
clc48grbk0031qsoadjbqc0g3     18
clb0rxjym0015qsf215jvzb4g     16
clb67qk5a0000qsoapsq2bksd     13
clakobin8000hqsc8mfqr1pg1     12
clb395a1x001bqsf29d38xolg     12
clbcyijtf000pqsoa0isf0p74     12
clc41tw1r002xqsoa0ttlirta     10
clash9cyo000tqsf20r5xjy2c     10
claobq8050009qsf2q0jlad2r      9
clbbsg0tn000jqsoa7kico3jl      8
clary8vpy000oqsf255p2o3fl      7
clbkd5w26001kqsoaf8h6199x      7
clc4fg3rj0036qsoabkdl82ng      7
clbhytybm001fqsoavbllmj80      5
clbfqkzb40

In [53]:
df3_nomap_1 = df3_nomap[(df3_nomap['kbm_apartment_id'].isnull() == False) & (df3_nomap['user_id'].isnull() == False)]
df3_nomap_1 = df3_nomap_1[['user_id', 'event_name','kbm_apartment_id']]
df3_nomap_1

,user_id,event_name,kbm_apartment_id
164,clba7eici000aqsoaangf9wpi,page_view,1372
168,clba7eici000aqsoaangf9wpi,page_view,8244
184,clba7eici000aqsoaangf9wpi,page_view,8244
188,clba7eici000aqsoaangf9wpi,page_view,14130
190,clba7eici000aqsoaangf9wpi,page_view,28620
...,...,...,...
11633,cl8wejtlj0000l309v0qc88kc,page_view,19763
11638,cl8wejtlj0000l309v0qc88kc,page_view,44285
11648,cl8vem63w0000mh09b8zbsclp,page_view,31983
11790,cl8vem63w0000mh09b8zbsclp,page_view,32148


In [54]:
df3_nomap_1['apt_view_cnt'] = df3_nomap_1.groupby(['user_id', 'kbm_apartment_id'])['event_name'].transform('count')
df3_nomap_1

,user_id,event_name,kbm_apartment_id,apt_view_cnt
164,clba7eici000aqsoaangf9wpi,page_view,1372,1
168,clba7eici000aqsoaangf9wpi,page_view,8244,2
184,clba7eici000aqsoaangf9wpi,page_view,8244,2
188,clba7eici000aqsoaangf9wpi,page_view,14130,1
190,clba7eici000aqsoaangf9wpi,page_view,28620,1
...,...,...,...,...
11633,cl8wejtlj0000l309v0qc88kc,page_view,19763,8
11638,cl8wejtlj0000l309v0qc88kc,page_view,44285,7
11648,cl8vem63w0000mh09b8zbsclp,page_view,31983,1
11790,cl8vem63w0000mh09b8zbsclp,page_view,32148,1


In [58]:
def area_jp(x):
    if x < 22 :
        area_div_nm = "22평미만"
    elif (x >= 22) & (x < 30):
        area_div_nm = "20평대"
    elif (x >= 30) & (x < 40):
        area_div_nm = "30평대"
    elif (x >= 40) & (x < 50):
        area_div_nm = "40평대"
    elif (x >= 50):
        area_div_nm = "50평대이상"
    
    return area_div_nm

In [59]:
kbm_apt_info["평형"] = kbm_apt_info["net_area"] / 3.305785
kbm_apt_info["평형대"] = kbm_apt_info["평형"].astype(int)
kbm_apt_info["평형구분"] = kbm_apt_info.apply(lambda x : area_jp(x['평형대']), axis = 1)
kbm_apt_info

,kbm_apartment_id,province,city,dong,lee,district_code,address,name,number_of_units_total,built_year_month,...,year_month,price_mid,jeonse_mid,price_per,price_per_g,rent_per,jeonse_rate,평형,평형대,평형구분
0,1,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 105,동현,548,198610,...,202211,210000,77500,8174.927579,6774.810676,3016.937559,36.904762,25.688301,25,20평대
1,1,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 105,동현,548,198610,...,202211,220000,100000,6031.453807,5159.792125,2741.569912,45.454545,36.475451,36,30평대
2,1,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 105,동현,548,198610,...,202211,232500,110000,5122.600723,4382.455311,2423.596041,47.311828,45.387102,45,40평대
3,1,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 105,동현,548,198610,...,202211,220000,100000,6077.318459,5133.931244,2762.417481,45.454545,36.200176,36,30평대
4,2,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 22,논현(신동아),644,199707,...,202211,97500,43000,9020.823887,6090.590278,3978.414638,44.102564,10.808325,10,22평미만
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91164,661804,부산광역시,해운대구,재송동,,2635010400,부산광역시 해운대구 재송동 1048-3,센텀우신골든빌,846,200610,...,202211,25500,18500,1414.387878,1026.391300,1026.124539,72.549020,18.029001,18,22평미만
91165,661804,부산광역시,해운대구,재송동,,2635010400,부산광역시 해운대구 재송동 1048-3,센텀우신골든빌,846,200610,...,202211,36000,20500,1564.662898,1243.165779,890.988595,56.944444,23.008151,23,20평대
91166,661804,부산광역시,해운대구,재송동,,2635010400,부산광역시 해운대구 재송동 1048-3,센텀우신골든빌,846,200610,...,202211,41000,23000,1597.562294,1310.170952,896.193482,56.097561,25.664101,25,20평대
91167,661804,부산광역시,해운대구,재송동,,2635010400,부산광역시 해운대구 재송동 1048-3,센텀우신골든빌,846,200610,...,202211,41000,23000,1605.890818,1301.490157,900.865581,56.097561,25.531001,25,20평대


In [60]:
kbm_per = kbm_apt_info.groupby(['province', 'city', 'dong', 'district_code','평형구분'])['price_per'].mean().reset_index()
kbm_per['class'] = pd.qcut(kbm_per['price_per'], q=500, labels = list(range(500, 0, -1)))
kbm_per['class'] = kbm_per['class'].astype(int)
kbm_per

,province,city,dong,district_code,평형구분,price_per,class
0,강원도,강릉시,견소동,4215011500,20평대,1084.785344,294
1,강원도,강릉시,견소동,4215011500,22평미만,1298.482751,242
2,강원도,강릉시,견소동,4215011500,30평대,818.122740,372
3,강원도,강릉시,교동,4215011000,20평대,1154.865700,278
4,강원도,강릉시,교동,4215011000,22평미만,1252.558294,253
...,...,...,...,...,...,...,...
6874,충청북도,충주시,칠금동,4313011600,50평대이상,493.121646,463
6875,충청북도,충주시,풍동,4313011000,22평미만,332.037735,493
6876,충청북도,충주시,호암동,4313010700,20평대,1353.805398,229
6877,충청북도,충주시,호암동,4313010700,22평미만,723.230764,402


In [62]:
kbm_per[kbm_per['class'] == 1]

,province,city,dong,district_code,평형구분,price_per,class
4161,서울특별시,강남구,개포동,1168010300,20평대,10464.060007,1
4163,서울특별시,강남구,개포동,1168010300,30평대,10837.250937,1
4199,서울특별시,강남구,압구정동,1168011000,20평대,14489.160837,1
4200,서울특별시,강남구,압구정동,1168011000,22평미만,15157.728835,1
4201,서울특별시,강남구,압구정동,1168011000,30평대,10815.649816,1
4202,서울특별시,강남구,압구정동,1168011000,40평대,10456.435867,1
4203,서울특별시,강남구,압구정동,1168011000,50평대이상,10946.190773,1
4621,서울특별시,서초구,반포동,1165010700,20평대,11866.427153,1
4622,서울특별시,서초구,반포동,1165010700,22평미만,11985.374472,1
4623,서울특별시,서초구,반포동,1165010700,30평대,12289.334164,1


In [63]:
kbm_per[kbm_per['class'] == 2]

,province,city,dong,district_code,평형구분,price_per,class
4171,서울특별시,강남구,대치동,1168010600,20평대,9882.128278,2
4173,서울특별시,강남구,대치동,1168010600,30평대,9583.082615,2
4174,서울특별시,강남구,대치동,1168010600,40평대,8713.052643,2
4175,서울특별시,강남구,대치동,1168010600,50평대이상,8597.300500,2
4176,서울특별시,강남구,도곡동,1168011800,20평대,8414.817457,2
4211,서울특별시,강남구,일원동,1168011400,20평대,8697.338031,2
4212,서울특별시,강남구,일원동,1168011400,22평미만,9317.948132,2
4218,서울특별시,강남구,청담동,1168010400,20평대,8785.629484,2
4646,서울특별시,서초구,잠원동,1165010600,22평미만,9027.679347,2
4647,서울특별시,서초구,잠원동,1165010600,30평대,8771.135036,2


In [66]:
kbm_apt_info['age'] = (pd.datetime.now() - pd.to_datetime(kbm_apt_info['built_year_month'], format = '%Y%m')) / np.timedelta64(1, 'Y')
kbm_apt_info

,kbm_apartment_id,province,city,dong,lee,district_code,address,name,number_of_units_total,built_year_month,...,jeonse_mid,price_per,price_per_g,rent_per,jeonse_rate,평형,평형대,평형구분,class,age
0,1,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 105,동현,548,198610,...,77500,8174.927579,6774.810676,3016.937559,36.904762,25.688301,25,20평대,10,36.245626
1,2,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 22,논현(신동아),644,199707,...,82500,6746.899107,5490.544122,3226.777834,47.826087,25.567301,25,20평대,10,25.496603
2,2,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 22,논현(신동아),644,199707,...,82500,6635.771047,4478.669509,3220.300655,48.529412,25.618726,25,20평대,10,25.496603
3,3,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 274,경남논현,60,199609,...,72500,5473.801301,4077.261034,2834.647102,51.785714,25.576376,25,20평대,10,26.326188
4,3,서울특별시,강남구,논현동,,1168010800,서울특별시 강남구 논현동 274,경남논현,60,199609,...,72500,5451.877724,4060.804598,2823.293821,51.785714,25.679226,25,20평대,10,26.326188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91164,353561,전라북도,군산시,오식도동,,4513014700,전라북도 군산시 오식도동 806-3,한성필하우스,892,201208,...,9500,689.279608,500.149026,523.852502,76.000000,18.134876,18,22평미만,415,10.410735
91165,353561,전라북도,군산시,오식도동,,4513014700,전라북도 군산시 오식도동 806-3,한성필하우스,892,201208,...,9500,689.279608,493.165205,523.852502,76.000000,18.134876,18,22평미만,415,10.410735
91166,385927,전라북도,정읍시,상평동,,4518010800,전라북도 정읍시 상평동 56-52,KB채움가,50,201905,...,14500,970.991589,725.603811,639.971729,65.909091,22.657251,22,20평대,330,3.664532
91167,385927,전라북도,정읍시,상평동,,4518010800,전라북도 정읍시 상평동 56-52,KB채움가,50,201905,...,14500,965.448958,717.868621,636.318631,65.909091,22.787326,22,20평대,330,3.664532


In [67]:
kbm_apt_info['apt_price_per'] = kbm_apt_info.groupby(['kbm_apartment_id'])['price_per'].transform('mean')
kbm_apt_info['apt_rent_per'] = kbm_apt_info.groupby(['kbm_apartment_id'])['rent_per'].transform('mean')
kbm_apt_info['jeonse_rate_mean'] = kbm_apt_info.groupby(['kbm_apartment_id'])['jeonse_rate'].transform('mean')
kbm_apt_info['class_apt'] = kbm_apt_info.groupby(['kbm_apartment_id'])['class'].transform('mean')

kbm_apt_info_res = kbm_apt_info[['kbm_apartment_id', 'age', 'class_apt','number_of_units_total', 'apt_price_per',  'apt_rent_per', 'jeonse_rate_mean']]
kbm_apt_info_res = kbm_apt_info_res.drop_duplicates()
kbm_apt_info_res

,kbm_apartment_id,age,class_apt,number_of_units_total,apt_price_per,apt_rent_per,jeonse_rate_mean
0,1,36.245626,15.25,548,6351.575142,2736.130248,43.781420
1,2,25.496603,20.75,644,7223.553090,3347.240350,46.528770
3,3,26.326188,18.60,60,5261.139191,2873.083276,54.707792
6,4,26.495939,27.50,111,5116.313939,2783.340319,54.488843
7,5,46.077450,10.00,70,5245.760044,1742.468899,33.281250
...,...,...,...,...,...,...,...
91135,312512,0.411899,183.00,431,1594.930395,1069.770387,67.073171
91136,340365,1.159347,257.00,600,1241.189375,855.041570,68.888889
91146,344349,1.663122,117.00,1331,2209.795616,1554.996366,70.371622
91162,353561,10.410735,415.00,892,682.376481,497.960140,72.878617


In [70]:
def cre_len(x):
    cnt = len(x) + 1
    return cnt

In [71]:
user_info['user_age'] = (pd.datetime.now() -pd.to_datetime(user_info['owner_birth_years'], format='%Y')) / np.timedelta64(1, 'Y')

NameError: name 'user_info' is not defined

In [ ]:
user_info['resident_1'] = (user_info['resident_birth_years'].astype(str).str.split(','))
user_info['resident_cnt'] = user_info.apply(lambda x : cre_len(x['resident_1']), axis = 1)
user_info = user_info.drop(columns=['resident_1', 'loan_amount', 'owner_birth_years','wealth_holding', 'household_income'])
user_info = user_info.drop_duplicates()
user_info

In [ ]:
kbm_apt_info_res.info()

In [ ]:
user_view_apt_all.info()

In [ ]:
user_view_apt_all['kbm_apartment_id'] = user_view_apt_all['kbm_apartment_id'].astype(int)
user_view_apt_all_res = pd.merge(user_view_apt_all, kbm_apt_info_res, on=['kbm_apartment_id'], how = 'inner')
user_view_apt_all_res

In [ ]:
profile_1 = user_view_apt_all_res.groupby(['user_id'])['apt_price_per'].mean().reset_index()
profile_1 = profile_1.rename(columns = {'apt_price_per' : 'price_per_mean'})
profile_1

In [ ]:
profile_1_1 = user_view_apt_all_res.groupby(['user_id'])['apt_rent_per'].mean().reset_index()
profile_1_1 = profile_1_1.rename(columns = {'apt_rent_per' : 'rent_per_mean'})
profile_1_1

In [ ]:
profile_2 = user_view_apt_all_res.groupby(['user_id'])['age'].mean().reset_index()
profile_2 = profile_2.rename(columns = {'age' : 'age_mean'})
profile_2

In [ ]:
profile_3 = user_view_apt_all_res.groupby(['user_id'])['number_of_units_total'].mean().reset_index()
profile_3 = profile_3.rename(columns = {'number_of_units_total' : 'number_of_units_total_mean'})
profile_3

In [ ]:
profile_4 = user_view_apt_all_res.groupby(['user_id'])['jeonse_rate_mean'].mean().reset_index()
profile_4 = profile_4.rename(columns = {'jeonse_rate_mean' : 'jeonse_rate_avg'})
profile_4

In [ ]:
profile_5 = user_view_apt_all.groupby(['user_id'])['user_apt_view_cnt'].sum().reset_index()
profile_5 = profile_5.rename(columns={'user_apt_view_cnt' : 'view_sum'})
profile_5

In [ ]:
profile_6 = user_view_apt_all.groupby(['user_id'])['kbm_apartment_id'].count().reset_index()
profile_6 = profile_6.rename(columns={'kbm_apartment_id' : 'view_apt_cnt'})
profile_6

In [ ]:
user_view_apt_all_res.columns

In [ ]:
profile_7 = user_view_apt_all_res.groupby(['user_id'])['class_apt'].mean().reset_index()
profile_7 = profile_7.rename(columns = {'class_apt' : 'class_apt_mean'})
profile_7

In [ ]:
df_time = df[['user_id', 'event_date', 'time']]
df_time = df_time[(df_time['user_id'].isnull()==False) & (df_time['time'].isnull() == False)]
df_time['date_cnt'] = df_time.groupby(['user_id'])['event_date'].transform('nunique')
df_time['used_time_sum'] = df_time.groupby(['user_id'])['time'].transform('sum')
df_time['user_time_per_day'] = df_time['used_time_sum'] / df_time['date_cnt']
df_time = df_time.drop(columns=['event_date', 'time'])
df_time = df_time.drop_duplicates()
df_time

In [ ]:
df_like = m_dt11[['user_id', 'LIKE']]
df_like['LIKE_cnt'] = df_like.groupby(['user_id'])['LIKE'].transform('count')
df_like = df_like.drop(columns=['LIKE'])
df_like = df_like.drop_duplicates()
df_like

In [ ]:
profile_all = pd.merge(profile_1, profile_2, on =['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, profile_1_1, on= ['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, profile_3, on =['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, profile_4, on =['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, profile_5, on =['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, profile_6, on =['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, profile_7, on =['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, df_time, on =['user_id'], how = 'inner')
profile_all = pd.merge(profile_all, df_like, on=['user_id'] , how = 'left')
profile_all = profile_all.fillna(0)
profile_all.columns = ['user_id', '평균 평당가(매매)','평균 연식', '평균 평당가(전세)', 
                       '평균 총세대수', '평균 전세가율', '이벤트(view) 발생 합계','조회한 단지수', '조회한 평균 급지',
                       '사용일(일/day)', '총 사용시간합(초/sec)','일별 평균 사용시간(초/sec)', '관심아파트 등록수']
profile_all

In [ ]:
user_view_apt_all_res['apt_price_per_min'] = user_view_apt_all_res.groupby(['user_id'])['apt_price_per'].transform('min')
user_view_apt_all_res['apt_price_per_max'] = user_view_apt_all_res.groupby(['user_id'])['apt_price_per'].transform('max')

user_view_apt_all_res['apt_rent_per_min'] = user_view_apt_all_res.groupby(['user_id'])['apt_rent_per'].transform('min')
user_view_apt_all_res['apt_rent_per_max'] = user_view_apt_all_res.groupby(['user_id'])['apt_rent_per'].transform('max')

user_view_apt_all_res['age_min'] = user_view_apt_all_res.groupby(['user_id'])['age'].transform('min')
user_view_apt_all_res['age_max'] = user_view_apt_all_res.groupby(['user_id'])['age'].transform('max')

user_view_apt_all_res['number_of_units_min'] = user_view_apt_all_res.groupby(['user_id'])['number_of_units_total'].transform('min')
user_view_apt_all_res['number_of_units_max'] = user_view_apt_all_res.groupby(['user_id'])['number_of_units_total'].transform('max')

user_view_apt_all_res['jeonse_rate_min'] = user_view_apt_all_res.groupby(['user_id'])['jeonse_rate_mean'].transform('min')
user_view_apt_all_res['jeonse_rate_max'] = user_view_apt_all_res.groupby(['user_id'])['jeonse_rate_mean'].transform('max')

user_view_apt_all_res
